In [51]:
import pandas as pd
import numpy as np

In [52]:
dataset = pd.read_csv("Anxiety.csv")

In [53]:
dataset

,1. Age,2. Gender,3. University,4. Department,5. Academic Year,6. Current CGPA,7. Did you receive a waiver or scholarship at your university?,"1. In a semester, how often you felt nervous, anxious or on edge due to academic pressure?","2. In a semester, how often have you been unable to stop worrying about your academic affairs?","3. In a semester, how often have you had trouble relaxing due to academic pressure?","4. In a semester, how often have you been easily annoyed or irritated because of academic pressure?","5. In a semester, how often have you worried too much about academic affairs?","6. In a semester, how often have you been so restless due to academic pressure that it is hard to sit still?","7. In a semester, how often have you felt afraid, as if something awful might happen?",Anxiety Value,Anxiety Label
0,18-22,Female,"Independent University, Bangladesh (IUB)",Engineering - CS / CSE / CSC / Similar to CS,Second Year or Equivalent,2.50 - 2.99,No,2,2,3,2,2,2,2,15,Severe Anxiety
1,18-22,Male,"Independent University, Bangladesh (IUB)",Engineering - CS / CSE / CSC / Similar to CS,Third Year or Equivalent,3.00 - 3.39,No,1,2,2,1,1,3,2,12,Moderate Anxiety
2,18-22,Male,American International University Bangladesh (...,Engineering - CS / CSE / CSC / Similar to CS,Third Year or Equivalent,3.00 - 3.39,No,0,0,0,0,0,0,0,0,Minimal Anxiety
3,18-22,Male,American International University Bangladesh (...,Engineering - CS / CSE / CSC / Similar to CS,Third Year or Equivalent,3.00 - 3.39,No,2,1,1,1,2,1,2,10,Moderate Anxiety
4,18-22,Male,North South University (NSU),Engineering - CS / CSE / CSC / Similar to CS,Second Year or Equivalent,2.50 - 2.99,No,3,0,3,3,1,1,3,14,Moderate Anxiety
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,18-22,Male,Dhaka University (DU),Other,Second Year or Equivalent,3.00 - 3.39,No,3,3,3,3,3,3,3,21,Severe Anxiety
2024,18-22,Female,Bangladesh Agricultural University (BAU),Engineering - CS / CSE / CSC / Similar to CS,Third Year or Equivalent,3.00 - 3.39,No,3,0,3,3,3,3,3,18,Severe Anxiety
2025,18-22,Male,American International University Bangladesh (...,Engineering - CS / CSE / CSC / Similar to CS,Second Year or Equivalent,2.50 - 2.99,No,2,2,2,2,2,2,1,13,Moderate Anxiety
2026,23-26,Female,Dhaka University (DU),Other,Third Year or Equivalent,3.40 - 3.79,No,3,3,3,3,3,3,3,21,Severe Anxiety


In [54]:
x = dataset.iloc[:,0:7]
y = dataset.iloc[:,-1]

In [55]:
y = y.map(lambda y: "Less Anxious" if y=="Minimal Anxiety" else "More Anxious");

In [56]:
y.value_counts()

More Anxious    1869
Less Anxious     159
Name: Anxiety Label, dtype: int64

In [16]:
class_ratio = y.value_counts()[0] / y.value_counts()[1]
y.value_counts()[0],y.value_counts()[1]
class_ratio

11.754716981132075

In [60]:
x_encoded = pd.get_dummies(x, drop_first=True,dtype=int)
y_encoded= pd.get_dummies(y,drop_first=True,dtype=int)
x_encoded
columns_for_prediction = x_encoded.columns
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42)
x_resampled, y_resampled = sm.fit_resample(x_encoded, y_encoded)


In [61]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_resampled,y_resampled,test_size=0.3,random_state=0)

In [62]:
y_train.value_counts()

More Anxious
0               1315
1               1301
dtype: int64

In [63]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=10,criterion="gini",random_state=0,class_weight='balanced')
random_forest.fit(x_train,y_train)

C:\Users\Vignesh S\AppData\Local\Temp\ipykernel_7112\2578238666.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest.fit(x_train,y_train)


RandomForestClassifier(class_weight='balanced', n_estimators=10, random_state=0)

In [64]:
top_features = pd.Series(random_forest.feature_importances_,index=x_train.columns).sort_values(ascending=False)

In [65]:
y_predict = random_forest.predict(x_test)

In [66]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_predict)

In [67]:
cm

array([[518,  36],
       [146, 422]], dtype=int64)

In [68]:
from sklearn.metrics import classification_report
report = classification_report(y_test,y_predict)
print(report)

              precision    recall  f1-score   support

           0       0.78      0.94      0.85       554
           1       0.92      0.74      0.82       568

    accuracy                           0.84      1122
   macro avg       0.85      0.84      0.84      1122
weighted avg       0.85      0.84      0.84      1122



In [69]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

estimator = {
    'criterion':['gini', 'entropy', 'log_loss'],
    'max_features':['sqrt', 'log2', None],
    'n_estimators':[50,100]

}

grid = GridSearchCV(RandomForestClassifier(),estimator,verbose=3,refit=True,n_jobs=-1)

In [70]:
re = grid.fit(x_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [71]:
re

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [50, 100]},
             verbose=3)

In [72]:
table = pd.DataFrame.from_dict(re.cv_results_)


In [73]:
print(f'BEST PARAMETERS: {grid.best_params_}')

BEST PARAMETERS: {'criterion': 'log_loss', 'max_features': 'log2', 'n_estimators': 50}


In [74]:
# university = int(input("Enter:\n1 for IUB\n2 for AIUB\n3 for NSU\n4 for IUT\n5 for PSTU\n6 for RUET\n7 for DU\n8 for BUET\n9 for DUET\n10 for UIU\n11 for EWU\n12 for BRAC\n13 for BAU\n14 for RU\n15 for Daffodil\n"))-1
entered_age = int(input("Enter your Age: "))
age=""
if(entered_age<18):
    age="Below 18"
elif entered_age<=22:
    age="18-22"
elif entered_age<=26:
    age="23-26"
elif entered_age<=30:
    age="27-30"
else:
    age="Above 30"
    
entered_gender = input("Gender (male/female/m/f): ").lower()[0:1]

if "m" in entered_gender:
    gender="Male"
elif "f" in entered_gender:
    gender="Female"
else:
    gender="Prefer not to say"
    
for i,value in enumerate(x["3. University"].unique()):
    
    print(f"Enter: {i+1} for {value}")
    
university = int(input("Enter Here:"))-1
    
selected_university = ""
for i,value in enumerate(x["3. University"].unique()):
    if(university==i):
        selected_university=value
        break;

        
for i,value in enumerate(x["4. Department"].unique()):
    
    print(f"Enter: {i+1} for {value}")
    
department = int(input("Enter Here:"))-1
    
selected_department = ""
for i,value in enumerate(x["4. Department"].unique()):
    if(department==i):
        selected_department=value
        break;
    
for i,value in enumerate(x["5. Academic Year"].unique()):
    
    print(f"Enter: {i+1} for {value}")
    
academic_year = int(input("Enter Here:"))-1
    
selected_academic_year = ""
for i,value in enumerate(x["5. Academic Year"].unique()):
    if(academic_year==i):
        selected_academic_year=value
        break;
        
for i,value in enumerate(x["6. Current CGPA"].unique()):
    
    print(f"Enter: {i+1} for {value}")
    
current_cgpa = int(input("Enter Here:"))-1
    
selected_current_cgpa = ""
for i,value in enumerate(x["6. Current CGPA"].unique()):
    if(current_cgpa==i):
        selected_current_cgpa=value
        break;
        
for i,value in enumerate(x["7. Did you receive a waiver or scholarship at your university?"].unique()):
    
    print(f"Enter: {i+1} for {value}")
    
entered_scholarship = int(input("Enter Here:"))-1
    
selected_scholarship = ""
for i,value in enumerate(x["7. Did you receive a waiver or scholarship at your university?"].unique()):
    if(entered_scholarship==i):
        selected_scholarship=value
        break;

    
print(f"You're Age:{entered_age}, You are grouped under the age group: {age}")
print(f"You're Gender:{gender}")  

print(f"You're Selected University:{selected_university}")
print(f"You're Selected Department:{selected_department}")  
print(f"You're Selected Academic Year:{selected_academic_year}")
print(f"You're CGPA:{current_cgpa}, You are grouped under the CGPA group: {selected_current_cgpa}")
print(f"You're Scholarship Status:{selected_scholarship}")

Enter your Age: 27
Gender (male/female/m/f): m
Enter: 1 for Independent University, Bangladesh (IUB)
Enter: 2 for American International University Bangladesh (AIUB)
Enter: 3 for North South University (NSU)
Enter: 4 for Islamic University of Technology (IUT)
Enter: 5 for Patuakhali Science and Technology University
Enter: 6 for Rajshahi University of Engineering and Technology (RUET)
Enter: 7 for Dhaka University (DU)
Enter: 8 for Bangladesh University of Engineering and Technology (BUET)
Enter: 9 for Dhaka University of Engineering and Technology (DUET)
Enter: 10 for United International University (UIU)
Enter: 11 for East West University (EWU)
Enter: 12 for BRAC University
Enter: 13 for Bangladesh Agricultural University (BAU)
Enter: 14 for Rajshahi University (RU)
Enter: 15 for Daffodil University
Enter Here:2
Enter: 1 for Engineering - CS / CSE / CSC / Similar to CS
Enter: 2 for Engineering - EEE/ ECE / Similar to EEE
Enter: 3 for Other
Enter: 4 for Business and Entrepreneurship S

In [75]:
new_data = {
    '1. Age': age,
    '2. Gender': gender,
    '3. University': selected_university,
    '4. Department': selected_department,
    '5. Academic Year': selected_academic_year,
    '6. Current CGPA': selected_current_cgpa,
    '7. Did you receive a waiver or scholarship at your university?': selected_scholarship
}

In [76]:
new_df = pd.DataFrame([new_data])
new_df_encoded = pd.get_dummies(new_df, dtype=int)
new_df_encoded = new_df_encoded.reindex(columns=columns_for_prediction, fill_value=0)
re.predict(new_df_encoded)

array([0])

In [77]:
from xgboost import XGBClassifier as xgb

In [78]:
xgbClass=xgb(n_estimators=100,max_depth=10,scale_pos_weight=class_ratio,max_leaves=10, verbosity=2, random_state=0)

In [79]:
xgbClass.fit(x_train,y_train)

[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\tree\updater_prune.cc:98: tree pruning end, 130 extra nodes, 0 pruned nodes, max_depth=10
[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\tree\updater_prune.cc:98: tree pruning end, 136 extra nodes, 0 pruned nodes, max_depth=10
[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\tree\updater_prune.cc:98: tree pruning end, 100 extra nodes, 0 pruned nodes, max_depth=10
[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\tree\updater_prune.cc:98: tree pruning end, 66 extra nodes, 0 pruned nodes, max_depth=10
[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c

[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\tree\updater_prune.cc:98: tree pruning end, 124 extra nodes, 0 pruned nodes, max_depth=10
[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\tree\updater_prune.cc:98: tree pruning end, 110 extra nodes, 0 pruned nodes, max_depth=10
[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\tree\updater_prune.cc:98: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth=10
[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\tree\updater_prune.cc:98: tree pruning end, 122 extra nodes, 0 pruned nodes, max_depth=10
[19:43:41] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=10,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=0, ...)

In [80]:
y_predict = xgbClass.predict(x_test)
xgbClass.score(x_test,y_test)


0.679144385026738

In [81]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_predict)

In [82]:
cm

array([[221, 333],
       [ 27, 541]], dtype=int64)

In [83]:
from sklearn.metrics import classification_report
report = classification_report(y_test,y_predict)
print(report)

              precision    recall  f1-score   support

           0       0.89      0.40      0.55       554
           1       0.62      0.95      0.75       568

    accuracy                           0.68      1122
   macro avg       0.76      0.68      0.65      1122
weighted avg       0.75      0.68      0.65      1122

